# Data Splits
We want to split the SynFerm data set into train, validation and test data.
For all splits, we will do 9 random repetitions.
For 1D and 2D split, which both use 3 different groups to split on, these will divide into 3 random repetitions for each of the 3 groups. 

### 0D Split
For the 0D split, we use a random train-test split.
We use a 80/10/10 split into train, val, and test set.

### 1D Split
For the 1D split, we use a (1D) GroupShuffleSplit.
Each individual split will be 70/15/15 train/test (of groups not samples!).
As groups, we use either initiator, monomer, or terminator.

### 2D Split
For the 2D split, we use a (2D) GroupShuffleSplit.
Each individual split will use 20% of groups as test set and 25% of remaining groups as validation set. 
Due to the dimensionality, this means we expect 0.2 * 0.2 = 4% of samples in the test and validation set and 0.800^2 * 0.75^2 = 36.0% of samples in the training set.
The remaining samples are not used to prevent leakage.
As groups, we use either \[initiator, monomer], \[monomer, terminator] or \[initiator, terminator].

### 3D Split
For the 3D split, we use a (3D) GroupShuffleSplit.
Each individual split will use 25% of groups as test set, 33% of remaining groups as validation set, and the remaining groups as training set.
Due to the dimensionality, this means we expect 0.25^3 = 1.5% of samples in the test and validation set and 0.75^3 * 0.67^3 = 12.5% of sample in the training set.

In [ ]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().resolve().parents[1]))

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

from src.definitions import DATA_DIR
from src.util.train_test_split import GroupShuffleSplitND
from util import write_indices_and_stats

In [ ]:
# Load data
data_filename = "synferm_dataset_2023-12-20_39486records.csv"
data_name = data_filename.rsplit("_", maxsplit=1)[0]
df = pd.read_csv(DATA_DIR / "curated_data" / data_filename)
df.shape

In [ ]:
df.head()

In [ ]:
# M_long_dia will be to sort diastereomers into the same group on group shuffle splits
diastereomers = {
    "Mon001": "Mon087",
    "Mon003": "Mon078",
    "Mon011": "Mon088",
    "Mon013": "Mon074",
    "Mon014": "Mon090",
    "Mon015": "Mon076",
    "Mon016": "Mon096",
    "Mon017": "Mon075",
    "Mon019": "Mon091",
    "Mon020": "Mon077",
    "Mon080": "Mon010",
}
df["M_long_dia"] = df["M_long"].replace(diastereomers)

## 0D split (not needed, see 0D_80 in truncated splits)

## 0D split final-retrain
To retrain the best model after selection, for the 0D split we only use one fold, split into training and validation set (used for early stopping of FFN training)

In [ ]:
splitter = ShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

indices = []
sizes = []
pos_class = []
for idx_train, idx_val in splitter.split(df):
    idx_test = []  # placeholder so we can use the write_indices_and_stats function, just delete fold0_test.csv later
    # add to list
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(), 

        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=0, 
    save_indices=True, 
    train_size="final_retrain"
)

## 1D split

In [ ]:
splitter = GroupShuffleSplit(n_splits=3, test_size=0.15, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.15/0.85, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

# note: for M_long, we need to obtain diastereomer relationships to sort them into the same group
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["M_long_dia"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["M_long_dia"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["T_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["T_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    train_size=""
)

## 1D split for Euan
Requirement: same split logic as above, but with 3x9 folds instead of 3x3 folds

In [ ]:
splitter = GroupShuffleSplit(n_splits=9, test_size=0.15, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplit(n_splits=1, test_size=0.15/0.85, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["I_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["I_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )

# note: for M_long, we need to obtain diastereomer relationships to sort them into the same group
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["M_long_dia"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["M_long_dia"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df["T_long"]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df["T_long"][idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=1, 
    save_indices=True, 
    train_size="euan27folds"
)

## 2D split

In [ ]:
splitter = GroupShuffleSplitND(n_splits=3, test_size=0.2, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.2/0.8, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long_dia"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long_dia"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["M_long_dia", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["M_long_dia", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
for idx_train_val, idx_test in splitter.split(list(range(len(df))), groups=df[["I_long", "T_long"]]):
    train, val = next(inner_splitter.split(idx_train_val, groups=df[["I_long", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
        
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=2, 
    save_indices=True, 
    train_size=""
)

## 3D split

In [ ]:
splitter = GroupShuffleSplitND(n_splits=9, test_size=0.25, random_state=np.random.RandomState(42))  # here, we reuse the outer splitter as well, so we use RandomState (not true, copyPaste error from before. Anyway, not a problem)
inner_splitter = GroupShuffleSplitND(n_splits=1, test_size=0.25/0.75, random_state=np.random.RandomState(42))  # we use a RandomState instance, not an int, because we will reuse this splitter several times

In [ ]:
indices = []
sizes = []
pos_class = []
for idx_train_val, idx_test in splitter.split(df, groups=df[["I_long", "M_long_dia", "T_long"]]):
    train, val = next(inner_splitter.split(df.iloc[idx_train_val], groups=df[["I_long", "M_long_dia", "T_long"]].iloc[idx_train_val]))
    # use indices to index indices :P (we need to obtain indices referring to the original dataframe)
    idx_train = idx_train_val[train]
    idx_val = idx_train_val[val]
    indices.append((idx_train, idx_val, idx_test))
    sizes.append((len(idx_train), len(idx_val), len(idx_test)))
    pos_class.append(
        (np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_train]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_val]).to_numpy(), 
         np.sum(df[['binary_A', 'binary_B', 'binary_C']].loc[idx_test]).to_numpy(),
        )
    )
    
print(sizes)
print(pos_class)

In [ ]:
write_indices_and_stats(
    indices, 
    sizes, 
    pos_class,
    total_size=len(df),
    data_name=data_name,
    split_dimension=3, 
    save_indices=True, 
    train_size=""
)

## Control: Check splits

### 1D split

In [ ]:
split_dimension = 1
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9): # only these are split on monomers
    
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 1D groups are mutually exclusive
    if fold_idx < 3: # first three are split on initiator
        assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["I_long"].iloc[train_idx], df["I_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["I_long"].iloc[val_idx], df["I_long"].iloc[test_idx])) == 0
    elif fold_idx < 6:  # next three are split on monomer
        assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["M_long"].iloc[train_idx], df["M_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long"].iloc[val_idx], df["M_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[train_idx], df["M_long_dia"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[train_idx], df["M_long_dia"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["M_long_dia"].iloc[val_idx], df["M_long_dia"].iloc[test_idx])) == 0
        
    else:  # last three are split on terminator
        assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[val_idx])) == 0
        assert len(np.intersect1d(df["T_long"].iloc[train_idx], df["T_long"].iloc[test_idx])) == 0
        assert len(np.intersect1d(df["T_long"].iloc[val_idx], df["T_long"].iloc[test_idx])) == 0


### 2D split

In [ ]:
split_dimension = 2
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 2D groups are mutually exclusive
    if fold_idx < 3: # first three are split on initiator and monomer
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia"]].iloc[val_idx]), np.unique(df[["I_long", "M_long_dia"]].iloc[test_idx]))) == 0
    elif fold_idx < 6:  # next three are split on monomer and terminator
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[train_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long", "T_long"]].iloc[val_idx]), np.unique(df[["M_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["M_long_dia", "T_long"]].iloc[val_idx]), np.unique(df[["M_long_dia", "T_long"]].iloc[test_idx]))) == 0
    else:  # last three are split on initiator and terminator
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[val_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0
        assert len(np.intersect1d(np.unique(df[["I_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "T_long"]].iloc[test_idx]))) == 0


### 3D split

In [ ]:
split_dimension = 3
split_dir = DATA_DIR / "curated_data" / "splits" / f"{data_name}_{split_dimension}D_split"
    
for fold_idx in range(9):
    # import indices
    train_idx = pd.read_csv(split_dir / f"fold{fold_idx}_train.csv")["index"].to_numpy()
    val_idx = pd.read_csv(split_dir / f"fold{fold_idx}_val.csv")["index"].to_numpy()
    test_idx = pd.read_csv(split_dir / f"fold{fold_idx}_test.csv")["index"].to_numpy()

    # check mutually exclusive
    assert len(np.intersect1d(train_idx, val_idx)) == 0
    assert len(np.intersect1d(train_idx, test_idx)) == 0
    assert len(np.intersect1d(val_idx, test_idx)) == 0

    # check 3D groups are mutually exclusive
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long", "T_long"]].iloc[test_idx]))) == 0
    
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[val_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[train_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[test_idx]))) == 0
    assert len(np.intersect1d(np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[val_idx]), np.unique(df[["I_long", "M_long_dia", "T_long"]].iloc[test_idx]))) == 0